In [1]:
from gensim.models import KeyedVectors
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load 500000 Nepali word2vec embeddings, taken from: https://github.com/rabindralamsal/Word2Vec-Embeddings-for-Nepali-Language/blob/master/README.md
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NepaliProject/SynonymReplacement/nepali_embeddings_word2vec.txt', binary=False)
#load cleaned train set
df_train = pd.read_csv('/content/drive/MyDrive/NepaliProject/SynonymReplacement/cleaned_train.csv')


In [4]:
# code to perform synonym replacement
#taken from the TextAugment library https://github.com/dsfsi/textaugment?tab=readme-ov-file#Word2vec-based-augmentation
#modified by removing .wv from "in self.model.wv.most_similar so that the code works with KeyedVectors
import gensim
import numpy as np
import random


class Word2vec:
    """
    A set of functions used to augment data.

    Typical usage: ::
        >>> from textaugment import Word2vec
        >>> t = Word2vec(model='path/to/gensim/model'or 'gensim model itself')
        >>> t.augment('I love school', top_n=10)
        i adore school
    """

    def __init__(self, **kwargs):
        """
        A method to initialize a model on a given path.
        :type random_state: int, float, str, bytes, bytearray
        :param random_state: seed
        :type model: str or gensim.models.word2vec.Word2Vec or gensim.models.fasttext.FastText
        :param model: The path to the model or the model itself.
        :type runs: int, optional
        :param runs: The number of times to augment a sentence. By default is 1.
        :type v: bool or optional
        :param v: Replace all the words if true. If false randomly replace words.
                Used in a Paper (https://www.cs.cmu.edu/~diyiy/docs/emnlp_wang_2015.pdf)
        :type p: float, optional
        :param p: The probability of success of an individual trial. (0.1<p<1.0), default is 0.5
        """

        # Set random state
        if 'random_state' in kwargs:
            self.random_state = kwargs['random_state']
            if isinstance(self.random_state, int):
                random.seed(self.random_state)
                np.random.seed(self.random_state)
            else:
                raise TypeError("random_state must have type int")

        # Set verbose to false if does not exists
        try:
            if kwargs['v']:
                self.v = True
            else:
                self.v = False
        except KeyError:
            self.v = False

        try:
            if "p" in kwargs:
                if type(kwargs['p']) is not float:
                    raise TypeError("p represent probability of success and must be a float from 0.1 to 0.9. E.g p=0.5")
                elif type(kwargs['p']) is float:
                    self.p = kwargs['p']
            else:
                kwargs['p'] = 0.5  # Set default value
        except KeyError:
            raise

        # Error handling of given parameters
        try:
            if "runs" not in kwargs:
                kwargs["runs"] = 1  # Default value for runs
            elif type(kwargs["runs"]) is not int:
                raise TypeError("DataType for 'runs' must be an integer")
            if "model" not in kwargs:
                raise ValueError("Set the value of model. e.g model='path/to/model' or model itself")
            if type(kwargs['model']) != str and 'gensim' not in str(type(kwargs['model'])).lower():
                raise TypeError("Model path must be a string, or the type of the model must be a gensim.models...")
        except (ValueError, TypeError):
            raise
        else:
            self.runs = kwargs["runs"]
            self.model = kwargs["model"]
            self.p = kwargs["p"]
            try:
                if type(self.model) is str:
                    self.model = gensim.models.Word2Vec.load(self.model)  # load word2vec or fasttext model
            except FileNotFoundError:
                print("Error: Model not found. Verify the path.\n")
                raise ValueError("Error: Model not found. Verify the path.")

    def geometric(self, data):
        """
        Used to generate Geometric distribution.

        :type data: list
        :param data: Input data

        :rtype:   ndarray or scalar
        :return:  Drawn samples from the parameterized Geometric distribution.
        """

        data = np.array(data)
        first_trial = np.random.geometric(p=self.p, size=data.shape[0]) == 1  # Capture success after first trial
        return data[first_trial]

    def augment(self, data: str, top_n: int = 10):
        """
        The method to replace words with similar words.

        :type data: str
        :param data: Input data
        :type top_n: int
        :param top_n: top_n of most similar words to randomly choose from

        :rtype:   str
        :return:  The augmented data
        """

        # Avoid nulls and other unsupported types
        if type(top_n) is not int:
            raise TypeError("Only integers are supported")
        if type(data) is not str:
            raise TypeError("Only strings are supported")
        # Lower case and split
        data_tokens = data.lower().split()

        # Verbose = True then replace all the words.
        if self.v:
            for _ in range(self.runs):
                for index in range(len(data_tokens)):  # Index from 0 to length of data_tokens
                    try:
                        similar_words = [syn for syn, t in self.model.most_similar(data_tokens[index], topn=top_n)] #I REMOVED .wv from "in self.model.wv.most_similar"
                        r = random.randrange(len(similar_words))
                        data_tokens[index] = similar_words[r].lower()  # Replace with random synonym from 10 synonyms
                    except KeyError:
                        pass  # For words not in the word2vec model
        else:  # Randomly replace some words
            for _ in range(self.runs):
                data_tokens_idx = [[x, y] for (x, y) in enumerate(data_tokens)]  # Enumerate data
                words = self.geometric(data=data_tokens_idx).tolist()  # List of words indexed
                for w in words:
                    try:
                        similar_words_and_weights = [(syn, t) for syn, t in self.model.most_similar(w[1])] #I REMOVED .wv from "in self.model.wv.most_similar"
                        similar_words = [word for word, t in similar_words_and_weights]
                        similar_words_weights = [t for word, t in similar_words_and_weights]
                        word = random.choices(similar_words, similar_words_weights, k=1)
                        data_tokens[int(w[0])] = word[0].lower()  # Replace with random synonym from 10 synonyms
                    except KeyError:
                        pass
            return " ".join(data_tokens)
        return " ".join(data_tokens)


In [5]:
#I will use word_replacer to augment sentences in the 'text' column of df_train
word_replacer= Word2vec(model=model)

word_replacer.augment('हजुर यो कुरा देश सबै जनता ले सुननु  परछ अब हुने चुनाव मा नेता हरु सनग हिसाब  मागनु  परछ')

'तिमि त्यसैगरी कुरा देश सबै आमजनता सुनाकोठी सुननु परछ अब हुन्छ चुनाव दाई नेतासहित देस सनग दृष्टि शौचमिन्द्रियनिग्रह अर्से'

In [6]:
#copy the entire datafram df_train
augmented_df_train = df_train.copy()
print(df_train.shape)
print(augmented_df_train.shape)


(5990, 3)
(5990, 3)


In [7]:
#to create an augmented train set
#perform synonym replacement on the entire 'text' column
augmented_df_train['text'] = augmented_df_train['text'].map(lambda text: word_replacer.augment(text))

In [11]:
print(df_train['text'][5550])
print(augmented_df_train['text'][5550])

पुलिसले के काम गर्छ सम्बन्धित मानिसलाई भेटेर पनि
पुलिसले के कामकाज गथ्र्यो संबन्धित मानिसलाई भेटेर पनि


In [12]:
augmented_df_train1 = augmented_df_train

In [16]:
#create a second augmented train set
augmented_df_train2 = df_train.copy()
augmented_df_train2['text'] = augmented_df_train2['text'].map(lambda text: word_replacer.augment(text))

In [17]:
print(df_train['text'][5500])
print(augmented_df_train1['text'][5500])
print(augmented_df_train2['text'][5500])

हजुर ५ मिनेट बोल्दा ५०० जना नेता हरु ले प्यान्ट मा मुट्छन
तिमि ५ मिनेट बोल्दा ५०० जना नेताकार्यकर्ता हरु टीकाभैरव कुर्ता हजुरबुवा मुट्छन
हजुर ५ मिनेट बोल्न ५०० पाँचजना नेतासँग यिनि बज्रबाराही प्यान्ट दाजु मुट्छन


In [18]:
#third augmented train set
augmented_df_train3 = df_train.copy()
augmented_df_train3['text'] = augmented_df_train3['text'].map(lambda text: word_replacer.augment(text))

In [19]:
print(df_train['text'][5500])
print(augmented_df_train3['text'][5500])

हजुर ५ मिनेट बोल्दा ५०० जना नेता हरु ले प्यान्ट मा मुट्छन
हजुर ५ मिनेट बोल्दै ५०० दुईजना नेतासँग संग ले प्यान्ट दाजु मुट्छन


In [20]:
#fourth augmented train set
augmented_df_train4 = df_train.copy()
augmented_df_train4['text'] = augmented_df_train4['text'].map(lambda text: word_replacer.augment(text))

In [21]:
print(df_train['text'][5500])
print(augmented_df_train4['text'][5500])

हजुर ५ मिनेट बोल्दा ५०० जना नेता हरु ले प्यान्ट मा मुट्छन
हजुर ५ मिनट बोलि ५०० जना नेता हरु ले प्यान्ट मा मुट्छन


In [22]:
#fifth augmented train set
augmented_df_train5 = df_train.copy()
augmented_df_train5['text'] = augmented_df_train5['text'].map(lambda text: word_replacer.augment(text))

In [23]:
print(df_train['text'][5500])
print(augmented_df_train5['text'][5500])

हजुर ५ मिनेट बोल्दा ५०० जना नेता हरु ले प्यान्ट मा मुट्छन
हजुर ५ मनेट बोल्दा ५०० नौजना नेताहरू हरु ले प्यान्ट भाइ मुट्छन


In [24]:
new_df_train = pd.concat([df_train, augmented_df_train1, augmented_df_train2, augmented_df_train3, augmented_df_train4, augmented_df_train5])

In [25]:
new_df_train.shape

(35940, 3)

In [31]:
#save new augmented train set as csv
new_df_train.to_csv('/content/drive/MyDrive/NepaliProject/Augmented/aug_train.csv', index=False, escapechar='\\') # Set escape character to backslash)

In [26]:
df_train.shape

(5990, 3)

In [27]:
augmented_df_train5.shape

(5990, 3)

In [29]:
35940/6

5990.0